
# 🏠 Apartment Monthly Trend — First Analysis Notebook
한국 아파트 **실거래 CSV**를 이용하여 **월별 평균 매매가 추세**를 빠르게 확인하는 노트북이다.

1) CSV 로드 (UTF-8 기본, 필요 시 EUC-KR 예시 제공)  
2) 전처리: `거래금액` 숫자화, 날짜 파생(`계약년`, `계약월`, `월`)  
3) 기초 통계 확인  
4) 월별 평균 가격 라인 차트 생성  
5) 특정 아파트/지역 템플릿  
6) 차트 파일 저장 함수


In [ ]:

# ✅ 필수 라이브러리
# - pandas: 테이블 데이터 처리
# - numpy: 수치 연산 (필요 시)
# - matplotlib.pyplot: 그래프 출력(라인차트)
# 주의: 본 노트북에서는 시각화에 matplotlib만 사용한다.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 노트북에서 그래프가 바로 보이도록
%matplotlib inline


In [ ]:

# ✅ 파일 경로 설정
# - 기본 샘플: data/sample_transactions.csv (UTF-8)
# - 공공데이터 CSV가 euc-kr인 경우 encoding 인자를 사용하면 된다.
FILE_PATH = "../data/sample_transactions.csv"

# (선택) EUC-KR 예시
# df = pd.read_csv(FILE_PATH, encoding="euc-kr")


In [ ]:

# ✅ CSV 로드
df = pd.read_csv(FILE_PATH)
print("행/열 크기:", df.shape)
df.head()


In [ ]:

# ✅ 전처리 1: '거래금액' 숫자화
# - '89,000' 같은 문자열을 정수형으로 변환한다.
# - errors='coerce'를 사용하여 문제가 있는 값은 NaN으로 처리 가능.
df['거래금액'] = (
    df['거래금액']
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.strip()
    .astype(int)
)


In [ ]:

# ✅ 전처리 2: 날짜 파생 컬럼
# - '계약년월'은 YYYYMM 형식이라고 가정한다.
# - '계약일'은 1~31 → 두 자리로 맞춘다.
df['계약년'] = df['계약년월'].astype(str).str[:4]
df['계약월'] = df['계약년월'].astype(str).str[4:6]
df['계약일'] = df['계약일'].astype(str).str.zfill(2)

# 월(YYYY-MM) 문자열 생성
df['월'] = df['계약년'].astype(str) + "-" + df['계약월'].astype(str)


In [ ]:

# ✅ 기초 확인
display(df.info())
display(df.describe())

# 가장 자주 거래된 아파트 TOP 10
top_apts = df['아파트'].astype(str).str.strip().value_counts().head(10)
display(top_apts)


In [ ]:

# ✅ 월별 평균 매매가 계산
monthly = df.groupby('월', as_index=True)['거래금액'].mean().sort_index()
monthly.head(12)


In [ ]:

# ✅ 월별 평균 매매가 라인차트
plt.figure(figsize=(12, 4))
plt.plot(monthly.index, monthly.values)
plt.xticks(rotation=45)
plt.title("월별 평균 아파트 매매가")
plt.ylabel("거래금액(만원)")
plt.tight_layout()
plt.show()


In [ ]:

# ✅ 특정 아파트 템플릿
# - 원하는 아파트명으로 바꿔서 사용
TARGET_APT = "역삼래미안"

apt_df = df[df['아파트'].astype(str).str.strip() == TARGET_APT].copy()
apt_monthly = apt_df.groupby('월', as_index=True)['거래금액'].mean().sort_index()

print(f"{TARGET_APT} 거래 건수:", len(apt_df))
display(apt_df.head())
display(apt_monthly.head())


In [ ]:

# ✅ 특정 아파트 월별 가격 변화 라인차트
if len(apt_monthly) > 0:
    plt.figure(figsize=(12, 4))
    plt.plot(apt_monthly.index, apt_monthly.values)
    plt.xticks(rotation=45)
    plt.title(f"{TARGET_APT} 월별 평균 매매가")
    plt.ylabel("거래금액(만원)")
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 해당 아파트 데이터가 부족하여 차트를 생략한다.")


In [ ]:

# ✅ 지역(구/동) 비교 템플릿
# - 'gu'와 'dong' 컬럼이 존재한다고 가정(샘플 CSV 포함)
# - 지역별 평균 매매가 TOP 10 확인
if {'gu','dong'}.issubset(df.columns):
    gu_avg = df.groupby('gu', as_index=True)['거래금액'].mean().sort_values(ascending=False)
    dong_avg = df.groupby('dong', as_index=True)['거래금액'].mean().sort_values(ascending=False)
    print("구 평균 TOP 10")
    display(gu_avg.head(10))
    print("동 평균 TOP 10")
    display(dong_avg.head(10))
else:
    print("⚠️ 'gu', 'dong' 컬럼이 없어 지역 비교를 생략한다.")


In [ ]:

# ✅ 차트 저장 함수 (선택)
# - charts/ 폴더가 없으면 자동 생성 후 PNG로 저장
import os

def save_plot(fig, filename: str, folder: str = "../charts"):
    os.makedirs(folder, exist_ok=True)
    path = os.path.join(folder, f"{filename}.png")
    fig.savefig(path, dpi=200, bbox_inches='tight')
    print("저장 완료:", path)


In [ ]:

# ✅ (예시) 월별 평균 매매가 차트를 PNG로 저장
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(monthly.index, monthly.values)
ax.set_title("월별 평균 아파트 매매가")
ax.set_ylabel("거래금액(만원)")
for label in ax.get_xticklabels():
    label.set_rotation(45)
fig.tight_layout()

save_plot(fig, "monthly_avg_price")



## ✔ 완료
확장 가능 포인트:

- 전세가/전세가율 결합 분석  
- 거래량(건수) 추세 및 상관관계 분석  
- 가격지수(부동산원)와의 비교  
- 지역별 클러스터링/시계열 예측 모델 추가
